# Projeto de PO-233 
Este projeto busca realizar análise do problema proposto pelo Data Science Challenge 2019 - ITA.

##### Alunos:
- Fernando Zanchitta
- Davi Xie
- Hugo Timóteo

##### Dados
Os dados podem ser adquiridos no site: https://www.kaggle.com/competitions/data-science-challenge-at-eef-2019/data



## Bibliotecas necessárias

In [52]:
import pandas as pd
import seaborn as sns
import numpy as np
import os
import matplotlib.pyplot as plt


### Leitura dos conjuntos de dados:
Os dados são lidos a partir dos arquivos $.csv$ gerados pelo _SubSampling.py_.

In [72]:
f_train = pd.read_csv('../dataset/features-training-subsampled.csv')
f_test = pd.read_csv('../dataset/features-test-subsampled.csv')
t_train = pd.read_csv('../dataset/target-training-subsampled.csv')

In [73]:
f_train.columns

Index(['id', 'property_id', 'created_on', 'operation', 'property_type',
       'place_name', 'place_with_parent_names', 'country_name', 'state_name',
       'geonames_id', 'lat_lon', 'lat', 'lon', 'currency',
       'surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',
       'expenses', 'description', 'title', 'image_thumbnail', 'collected_on'],
      dtype='object')

# Limpeza e Organização

Para começar a análise exploratória dos dados deve ser feita a retirada de atributos e informações que não serão utilizadas pelo projeto, a fins de simplificação e coerência com o proposto pela disciplina.

Nesse sentido vamos:
- Retirar as colunas irrelevantes
- Retirar as colunas de id
- Retirar colunas redundantes

#### Colunas irrelevantes

In [74]:
for column in f_train.columns:
    print(column, ' : ', f_train[column].nunique())



id  :  855706
property_id  :  649924
created_on  :  1262
operation  :  1
property_type  :  4
place_name  :  5078
place_with_parent_names  :  6549
country_name  :  1
state_name  :  28
geonames_id  :  0
lat_lon  :  95074
lat  :  92764
lon  :  91536
currency  :  4
surface_total_in_m2  :  2380
surface_covered_in_m2  :  1723
floor  :  230
rooms  :  36
expenses  :  3757
description  :  582283
title  :  401279
image_thumbnail  :  613700
collected_on  :  38


Vamos verificar os valores de algumas colunas para investigar a retirada:

In [75]:
# select unique values of columns that contain less that 30 unique values:
filtered_columns = f_train.columns[f_train.nunique() < 30]

# Get unique values for filtered columns
unique_values = f_train[filtered_columns].apply(lambda x: x.unique()).to_dict()

# Print unique values for each column
for column, values in unique_values.items():
    print(column, ' : ', values)

operation  :  ['sell']
property_type  :  ['apartment' 'house' 'PH' 'store']
country_name  :  ['Brasil']
state_name  :  ['São Paulo' 'Rio de Janeiro' 'Rio Grande do Sul' 'Goiás' 'Paraná'
 'Paraíba' 'Santa Catarina' 'Mato Grosso do Sul' 'Minas Gerais' 'Ceará'
 'Distrito Federal' 'Mato Grosso' 'Bahia' 'Espírito Santo' 'Pernambuco'
 'Rio Grande do Norte' 'Alagoas' 'Amazonas' 'Piauí' 'Pará' 'Tocantins'
 'Sergipe' 'Maranhão' 'Rondônia' 'Outros países' 'Amapá' 'Acre' 'Roraima']
geonames_id  :  [nan]
currency  :  ['BRL' 'MXN' 'USD' 'COP']


Podemos retirar os valores de _operation_, _country_name_ e _geonames_id_

In [76]:
f_train = f_train.drop(['operation','geonames_id','country_name'],axis=1)
f_test = f_test.drop(['operation','geonames_id','country_name'],axis=1)

#### Valores identificadores

Alguns atributos possuem valores únicos para cada evento, portanto não adicionam poder preditivo ao modelo. 

In [77]:
#check max count of each value in features:
# verify only non nan values
for column in f_train.columns:
    print(column, ' : ', f_train[column].value_counts().max())
    #print value that has max count
    print(f_train[column].value_counts().idxmax())

id  :  1
8082860
property_id  :  8
340a423d3a24f43703df3b470a98f8b5e2a813ca
created_on  :  54777
2017-05-23
property_type  :  556223
apartment
place_name  :  40420
Praia Grande
place_with_parent_names  :  40336
|Brasil|São Paulo|Praia Grande|
state_name  :  602997
São Paulo
lat_lon  :  4189
-30.0,-51.0
lat  :  4189
-30.0
lon  :  4532
-51.0
currency  :  855531
BRL
surface_total_in_m2  :  11919
0.0
surface_covered_in_m2  :  15250
70.0
floor  :  4608
1.0
rooms  :  254288
1.0
expenses  :  8103
500.0
description  :  22434
Excelente Oportunidade!Entre em contato e agende já uma visita com um de nossos corretores. Os preços, valores e caraterísticas exibidos poderão sofrer mudanças sem aviso prévio, por este motivo todas as informações deverão ser confirmadas pelo departamento comercial. Para otimizar o seu atendimento tenha em mãos o número da referência do imóvel (código). Entre em contato preenchendo o formulário e informe sempre um telefone para receber todas as informações.
title  :  513

Podemos retirar os atributos: _id_ e _property_id_, pois se comportam como identificadores.

In [78]:
f_train = f_train.drop(['id','property_id'],axis=1)
f_test = f_test.drop(['id','property_id'],axis=1)

Por decisão de projeto, vamos retirar a coluna de _image_thumbnail_, uma vez que nossa proposta não é trabalhar com imagens.

In [79]:
f_train = f_train.drop(['image_thumbnail'],axis=1)
f_test = f_test.drop(['image_thumbnail'],axis=1)

In [80]:
f_train.columns

Index(['created_on', 'property_type', 'place_name', 'place_with_parent_names',
       'state_name', 'lat_lon', 'lat', 'lon', 'currency',
       'surface_total_in_m2', 'surface_covered_in_m2', 'floor', 'rooms',
       'expenses', 'description', 'title', 'collected_on'],
      dtype='object')

#### Valores faltantes

In [81]:
# count nan values for each feature, percentage of nan values and type of feature:        
missing_values = pd.DataFrame(data={
    'Feature_name': f_train.columns,
    'missing_values': f_train.isnull().sum(),
    'percentage': f_train.isnull().sum() / len(f_train) * 100,
    'type': f_train.dtypes
})

In [82]:
missing_values.loc[missing_values['percentage']>0]

,Feature_name,missing_values,percentage,type
lat_lon,lat_lon,431168,50.387399,object
lat,lat,431168,50.387399,float64
lon,lon,431167,50.387283,float64
surface_total_in_m2,surface_total_in_m2,648624,75.799866,float64
surface_covered_in_m2,surface_covered_in_m2,203874,23.825239,float64
floor,floor,818281,95.626418,float64
rooms,rooms,293755,34.328963,float64
expenses,expenses,570255,66.641463,float64
description,description,2,0.000234,object


Técnicas de _Missing Values Inputation_ funcionam bem somente quando a maioria dos valores está presente. Nesse caso, podemos retirar alguns atributos cuja presença de valores faltantes excede $50\%$

In [83]:
columns_to_drop = missing_values.loc[missing_values['percentage']>50].index
f_train = f_train.drop(columns_to_drop,axis=1)
f_test = f_test.drop(columns_to_drop,axis=1)

O restante do conjunto de dados que iremos trabalhar é:

In [84]:
f_train.head()

,created_on,property_type,place_name,place_with_parent_names,state_name,currency,surface_covered_in_m2,rooms,description,title,collected_on
0,2017-05-05,apartment,Praia Grande,|Brasil|São Paulo|Praia Grande|,São Paulo,BRL,65.0,1.0,"Apartamento de 2 dorms, sendo 1 suíte com vara...",Apartamento 2 dorms na Guilhermina,2017-06
1,2014-07-21,apartment,Vila Leopoldina,|Brasil|São Paulo|São Paulo|Vila Leopoldina|,São Paulo,BRL,75.0,2.0,Nao informado,Nao informado,2015-01
2,2015-08-15,house,Paulínia,|Brasil|São Paulo|Paulínia|,São Paulo,BRL,185.0,2.0,"Casa Térrea, localizada no condomínio campos d...","Casa residencial à venda, Condomínio Campos d...",2015-09
3,2017-04-16,house,São Francisco,|Brasil|Rio de Janeiro|Niterói|São Francisco|,Rio de Janeiro,BRL,176.0,NaN,Atualizado em 05/06/2017. Excelente casa linea...,"Casa com 3 quartos e Jardim, Niterói, São Fran...",2017-06
4,2015-09-23,apartment,Vila Olímpia,|Brasil|São Paulo|São Paulo|Vila Olímpia|,São Paulo,BRL,127.0,1.0,Condomínio com ampla área de lazer : piscina a...,condominio com ampla area de lazer!!!,2015-11


Por uma decisão de projeto, não iremos realizar o processamento de linguagem natural presente no atributo _description_ e _title_. Dessa forma iremos descartá-lo,

In [85]:
f_train = f_train.drop(['description','title'],axis=1)
f_test = f_test.drop(['description','title'],axis=1)

In [87]:
f_train.head()

,created_on,property_type,place_name,place_with_parent_names,state_name,currency,surface_covered_in_m2,rooms,collected_on
0,2017-05-05,apartment,Praia Grande,|Brasil|São Paulo|Praia Grande|,São Paulo,BRL,65.0,1.0,2017-06
1,2014-07-21,apartment,Vila Leopoldina,|Brasil|São Paulo|São Paulo|Vila Leopoldina|,São Paulo,BRL,75.0,2.0,2015-01
2,2015-08-15,house,Paulínia,|Brasil|São Paulo|Paulínia|,São Paulo,BRL,185.0,2.0,2015-09
3,2017-04-16,house,São Francisco,|Brasil|Rio de Janeiro|Niterói|São Francisco|,Rio de Janeiro,BRL,176.0,NaN,2017-06
4,2015-09-23,apartment,Vila Olímpia,|Brasil|São Paulo|São Paulo|Vila Olímpia|,São Paulo,BRL,127.0,1.0,2015-11


# Análise Exploratória dos dados

Vamos analisar a coluna de descrição:

In [51]:
f_train.loc[220,'description']

'Excelente Apto, com 2 dormitórios, 1 banheiro, prontinho para mudar!!  Sala em L com dois ambientes, piso laminado e teto com moldura de gesso.  Primeiro dormitório com piso laminado, teto com moldura de gesso e armários planejados.  Segundo dormitório com piso laminado, teto com moldura de gesso e armário embutido.  Banheiro social com piso de cerâmica, teto com moldura de gesso, pia de madeira com cuba sobreposta de porcelana, gabinete e box de vidro.  Cozinha com piso de cerâmica, pia de granito com gabinete e armários planejados.  Área de serviço com piso de cerâmica e armários planejados.  Brinquedoteca equipada  Salão de Festas equipado  Condomínio novo com 5 anos  Localização com acesso fácil para Rod Anchieta.   Vizinhança tranquila.  Bairro arborizado.  Próximo ás faculdades: UNIP (700 mts) / ANHANGUERA (2km).  Próximo á comércios e escolas: mercados, hipermercados (Extra/Carrefour), farmácias, escolas (colégio objetivo), entre outros'

# Engenharia de Atributos

# Escolha de Atributos

# Treinamento e Teste

# Avaliação do Modelo

# Considerações Finais